# Conditional VAE

With Input:
- Image Label
- Coordinate

In [ ]:
import os
import csv
import numpy as np
import math
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from skimage import io
from PIL import Image
from tqdm import tqdm
#import argparse

import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.utils import save_image

matplotlib.style.use('ggplot')

torch.cuda.empty_cache() 
#import model

import ast

## To load in the dataset

In [ ]:
class ActiveVisionDataset (Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file, index_col=None)
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self,index):
        if type(index) == torch.Tensor:
            index = index.item()
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        #image = image/(image.max()/255.0)
        shape_label = torch.tensor(int(self.annotations.iloc[index,1]))
        #print(shape_label)
        cam_loc = torch.tensor(ast.literal_eval(self.annotations.iloc[index,2]))
        #print(cam_loc)
        if self.transform:
            image = self.transform(image)
        
        return image, shape_label, cam_loc

## Encoder

In [ ]:
class Encoder(nn.Module):
    def __init__(self, z_dim):
        super(Encoder, self).__init__()
        
        self.conv1 = nn.Conv2d(
            in_channels=3, out_channels=init_kernel, kernel_size=kernel_size, stride=stride_enc, padding=padding
        )
        self.conv2 = nn.Conv2d(
            in_channels=init_kernel, out_channels=init_kernel*2, kernel_size=kernel_size, stride=stride_enc, padding=padding
        )
        self.conv3 = nn.Conv2d(
            in_channels=init_kernel*2, out_channels=init_kernel, kernel_size=kernel_size, stride=stride_enc, padding=padding
        )

        self.img_lin1 = nn.Linear(init_kernel*(conv_out_size**2), 1024-9)

        self.lin1 = nn.Linear(1024,512)
        self.lin2 = nn.Linear(512,256)

        self.mu = nn.Linear(256, z_dim)
        self.sigma = nn.Linear(256, z_dim)
        
        self.dropout = nn.Dropout(p=dropout_pcent)
        
    def forward(self, image, label, coord):
        
        #Image                                                                 #print("before anything") #print(image.shape)
        x = self.conv1(image)
        x = F.relu(x)
        
        #print("before flatten:" + str(x.shape))
        x = x.view(x.size(0), -1)
        #print("after flatten:" + str(x.shape))

        x = self.img_lin1(x)
        x = F.relu(x)
        
        #Label and Coordinate                                                  #label = torch.unsqueeze(label, dim=1)
        label = F.one_hot(label, num_classes=6)
        y = torch.cat([coord,label],dim=1)
        x = torch.cat([x,y],dim=1)
                                                                               #print(label) print(label.shape) print(coord) 
                                                                 #print(coord.shape) print(y) print(y.shape) #print(x.shape)
        x = self.dropout(x)
        x = self.lin1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.lin2(x)
        x = F.relu(x)
        
        # get `mu` and `log_var`
        mu = self.mu(x)
        log_var = self.sigma(x)
        
        return mu, log_var

## Decoder

In [ ]:
class Decoder(nn.Module):
    def __init__(self, z_dim):
        super(Decoder, self).__init__()
        
        self.img_lin1 = nn.Linear(z_dim, 256-9)
        
        self.lin1 = nn.Linear(256,512)
        self.lin2 = nn.Linear(512,1024)
        self.lin3 = nn.Linear(1024, init_kernel*(conv_out_size**2))
        
        
        self.dec1 = nn.ConvTranspose2d(
            in_channels=init_kernel, out_channels=3, kernel_size=kernel_size, stride=stride_dec, padding=padding
        )
        self.dec2 = nn.ConvTranspose2d(
            in_channels=init_kernel*8, out_channels=init_kernel*4, kernel_size=kernel_size, stride=stride_dec, padding=padding
        )
        self.dec3 = nn.ConvTranspose2d(
            in_channels=init_kernel*4, out_channels=3, kernel_size=kernel_size, stride=stride_dec, padding=padding
        )
        
        self.dropout = nn.Dropout(p=dropout_pcent)
        
    def forward(self, z, label, coord):
        
        x = self.img_lin1(z)
        
        #Label and Coordinate
        #label = torch.unsqueeze(label, dim=1)
        label = F.one_hot(label, num_classes=6)
        y = torch.cat([coord,label],dim=1)
        x = torch.cat([x,y],dim=1)
        #print(x.shape)
        
        x = self.dropout(x)
        x = self.lin1(x)
        x=F.relu(x)
        
        x = self.dropout(x)
        x = self.lin2(x)
        x=F.relu(x)
        
        x = self.dropout(x)
        x = self.lin3(x)
        x=F.relu(x)
        
        x=x.view(-1, init_kernel, conv_out_size, conv_out_size)
        #print("after unflatten:")
        #print(x.shape)
        
        x = self.dec1(x)
        #x = F.relu(x)
        #x = self.dec2(x)
        #x = F.relu(x)
        #x = self.dec3(x)
        reconstruction = torch.sigmoid(x)
        
        return reconstruction

## VAE

In [ ]:
class ConditionalVAE(nn.Module):
    def __init__(self, z_dim):
        super(ConditionalVAE, self).__init__()
        self.encoder = Encoder(z_dim)
        self.decoder = Decoder(z_dim)
    
    def forward(self, image, label, coord):
        mu, log_var = self.encoder(image, label, coord)
        
        #print('mu: ', mu.shape)
        #print('log_var: ', log_var.shape)
        
        #sample z from latent distribution q
        std = torch.exp(log_var / 2)
        q = torch.distributions.Normal(mu,std)
        z = q.rsample()
        #print('z shape: ', z.shape)
        
        reconstruction = self.decoder(z, label, coord)
                
        return reconstruction, mu, log_var, z

## Loss Helper Functions

In [ ]:
def gaussian_likelihood(mean, logscale, sample):
    scale = torch.exp(logscale)
    dist = torch.distributions.Normal(mean, scale)
    log_pxz = dist.log_prob(sample)
    return log_pxz.sum(dim=(1, 2, 3))

def kl_divergence(z, mu, std):
    # --------------------------
    # Monte carlo KL divergence
    # --------------------------
    # 1. define the first two probabilities (in this case Normal for both)
    p = torch.distributions.Normal(torch.zeros_like(mu), torch.ones_like(std))
    q = torch.distributions.Normal(mu, std)

    # 2. get the probabilities from the equation
    log_qzx = q.log_prob(z)
    log_pz = p.log_prob(z)

    # kl
    kl = (log_qzx - log_pz)
    kl = kl.sum(-1)
    return kl

## Training

In [ ]:
def fit(model, dataloader):
    model.train()
    torch.set_grad_enabled(True) #???
    running_loss = 0.0
    
    for batch in tqdm(dataloader):
        image, label, coord = batch
        #print(image.size())
        #print(label)
        if torch.cuda.is_available():
            image = image.to(device)
            label = label.to(device)
            coord = coord.to(device)
        optimizer.zero_grad()
        
        reconstruction, mu, log_var, z = model(image, label, coord)
        
        #print(reconstruction.shape)
        
        #image = image.to(torch.device('cpu'))
        recon_loss = gaussian_likelihood(reconstruction, log_scale, image)
        
        std = torch.exp(log_var / 2)
        kl = kl_divergence(z, mu, std)

        elbo = (kl - recon_loss)
        elbo = elbo.mean()
        
        elbo.backward()
        optimizer.step()
        
        running_loss += elbo
    
    train_loss = running_loss/len(dataloader.dataset) #Investigate
    return train_loss

## Validation

In [ ]:
def validate(model, dataloader, epoch):
    model.eval()
    running_loss = 0.0
    i = 0
    with torch.no_grad():
        for batch in tqdm(dataloader):
            
            image, label, coord = batch
            
            if torch.cuda.is_available():
                image = image.to(device)
                label = label.to(device)
                coord = coord.to(device)
                
            reconstruction, mu, log_var, z = model(image, label, coord)
            
            if (i == int(len(val_data)/dataloader.batch_size) - 1 and ( ((epoch%val_img_out_freq))==0) ): # or epoch > 90
                num_rows = 4
                both = torch.cat((image.view(batch_size, 3, 128, 128)[:8], 
                                  reconstruction.view(batch_size, 3, 128, 128)[:8]))
                save_image(both.cpu(), f"outputs/{parameter}{value}/imgs/output{epoch}.png", nrow=num_rows)
            
            recon_loss = gaussian_likelihood(reconstruction, log_scale, image)
            
            std = torch.exp(log_var / 2)
            kl = kl_divergence(z, mu, std)

            elbo = (kl - recon_loss)
            elbo = elbo.mean()
            
            running_loss += elbo 
            
            i+=1
    
    val_loss = running_loss/len(dataloader.dataset)
    return val_loss

In [ ]:
def generate_latent_vectors(model, dataloader):
    model.eval()
    latent = []
    target = []
    with torch.no_grad():
        for batch in tqdm(dataloader):
            image, label, coord = batch
            #if torch.cuda.is_available():
            #    data = data.to(device)
            mu, logvar = model.encoder(image.cuda(), label.cuda(), coord.cuda())
            latent.extend(mu.cpu().detach().numpy())
            target.extend(label.numpy())
#         print(len(latent))
#         print(latent)
#         print(len(target))
#         print(target)
        return latent, target

## Run

In [ ]:
from IPython.display import clear_output
from time import sleep
def run_each():
    train_loss = []
    val_loss = []
    for epoch in range(epochs):
        clear_output(wait=True)
        print(f"Epoch {epoch+1} of {epochs}")
        sleep(0.2)
        train_epoch_loss = fit(model, train_loader)
        val_epoch_loss = validate(model, val_loader, epoch)

        train_loss.append(train_epoch_loss)
        val_loss.append(val_epoch_loss)

#         print(f"Train Loss: {train_epoch_loss:.4f}")
#         print(f"Val Loss: {val_epoch_loss:.4f}")
    return train_loss, val_loss

In [ ]:
def runall():
    train_loss, val_loss = run_each()
    plt.figure(figsize=(10,10))
    plt.plot(range(1,epochs+1), train_loss, label="Train Loss")
    plt.plot(range(1,epochs+1), val_loss, label="Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    axes = plt.gca()
    
    latent, target = generate_latent_vectors(model, val_loader)
    
    with open('outputs/'+parameter+value+'/sample_latent_vectors'+parameter+value+'.csv','w', newline='') as f:
        wr = csv.writer(f)
        wr.writerow(["Latent", "Target"])
        
        for i in range (0,len(latent)):
            latent[i] = list(latent[i])
            
        wr.writerows(zip(latent, target))
    
    filepath = os.path.join(os.getcwd(), "outputs", parameter+str(value), parameter+str(value)+".pth")
    torch.save(model.state_dict(), filepath)
    
    plt.savefig('outputs/'+parameter+value+'/loss'+parameter+value+'.png')
    
    with open('outputs/'+parameter+value+'/loss'+parameter+value+'.csv','w', newline='') as f:
        wr = csv.writer(f)
        wr.writerow(["Train loss", "Val loss"])
        
        for i in range (0,len(train_loss)):
            train_loss[i] = train_loss[i].item()
        
        for i in range (0,len(val_loss)):
            val_loss[i] = val_loss[i].item()
            
        wr.writerows(zip(train_loss, val_loss))
        
    with open('outputs/lossCompare.csv', 'a+', newline='') as f:
        wr = csv.writer(f)
        wr.writerow([parameter, value ,train_loss[-1], val_loss[-1], 
                     enc_out_dim, 
                     latent_dim, 
                     epochs,
                     batch_size, 
                     learning_rate, 
                     kernel_size, 
                     stride_enc,
                     stride_dec,
                     padding, 
                     init_kernel,
                     dropout_pcent,
                    ])

### Parameters

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

param_dict = {
    
    "enc_out_dim": 512,
    "latent_dim": 128,
    "conv_out_size": 63,
    
    "epochs" : 2,
    "batch_size" : 4,
    "learning_rate" : 0.001,
    
    "kernel_size" : 4,
    "stride_encoding" : 2,
    "stride_decoding" : 2, ###########################################################
    "padding" : 0,
    
    "init_kernel" : 8,
    
    "dropout_pcent": 0.4
    
}

In [ ]:
val_img_out_freq = 5
parameter_list = ['LatentStorageTest']
value_list = [1]

for parameter in parameter_list:
    for value in value_list:
        #Update the values to be updated and rerun the experiment
        enc_out_dim = param_dict["enc_out_dim"]
        latent_dim = param_dict["latent_dim"]
        conv_out_size = param_dict["conv_out_size"]

        epochs = param_dict["epochs"]
        batch_size = param_dict["batch_size"]
        learning_rate = param_dict["learning_rate"]

        kernel_size = param_dict["kernel_size"]
        stride_enc = param_dict["stride_encoding"]
        stride_dec = param_dict["stride_decoding"]
        padding = param_dict["padding"]
        init_kernel = param_dict["init_kernel"]
        
        dropout_pcent = param_dict["dropout_pcent"]
        
        #Move batch_size to before so its trained on the same split?
        set_used = 'fib_recognisable_imgs'
        train_data = ActiveVisionDataset(csv_file=set_used+'/TrainSet/rgbCSV.csv', root_dir=set_used+'/TrainSet/segImg/', transform = torchvision.transforms.ToTensor())
        val_data = ActiveVisionDataset(csv_file=set_used+'/ValSet/rgbCSV.csv', root_dir= set_used+'/ValSet/segImg/', transform = torchvision.transforms.ToTensor())
        train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(dataset=val_data, batch_size=batch_size, shuffle=True)

        img_size = len(train_data[0][0][0])
        model = ConditionalVAE(latent_dim).to(device)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        log_scale = nn.Parameter(torch.Tensor([0.0])).to(device)
        
        
        #Change the value to a string for later
        value = str(value)
        os.makedirs("outputs/"+parameter+str(value), exist_ok=True)
        os.makedirs("outputs/"+parameter+str(value)+"/imgs", exist_ok=True)
        
        #Run the Test
        runall()